# Modelo de predicción de inscripciones estudiantiles mediante técnicas de Machine Learning

**Maestría en Inteligencia Artificial Aplicada**

**Proyecto Integrador Sep-Nov 2025**

**Avance 2. Ingeniería de características**

**Equipo 14**

**Integrantes:**

- Alejandro Roa Solis – A01129942
- Annette Cristina Narvaez Andrade – A00571041
- Karla Alejandra Fonseca Márquez – A01795313


**Patrocinador Tec de Monterrey:**

Dr. Juan Arturo Nolazco Flores, Director del Hub de Ciencias y Datos de la Escuela de Ingeniería del Tec de Monterrey

En esta segunda etapa del **Proyecto Integrador**, enfocada en la *Ingeniería de características*, se busca optimizar el conjunto de datos con el fin de mejorar el rendimiento y la capacidad predictiva de los modelos de **atracción de estudiantes** para la universidad.  

El objetivo general del proyecto es comprender los factores que influyen en la decisión de inscripción de los prospectos y desarrollar modelos que permitan anticipar su comportamiento para fortalecer las estrategias de captación.  

Con este propósito, se decidió **dividir el dataset en tres subconjuntos**:  
1. Aspirantes de **entrada a Prepa Tec**,  
2. Estudiantes que **ingresan a universidad siendo egresados de Prepa Tec**, y  
3. Aspirantes que **ingresan a universidad sin haber cursado Prepa Tec**.  

Esta segmentación permite abordar de forma más precisa las particularidades de cada grupo y aplicar técnicas de *ingeniería de características* específicas —como **codificación, normalización, discretización y selección de variables**— que contribuyan a la construcción de modelos más robustos, interpretables y ajustados al contexto institucional.


## Resumen general de datos combinados

**Total de registros:** 40,780  
**Total de admitidos:** 23,010  

---

### Distribución por nivel académico (sin solapamientos)

| Nivel académico | Registros totales | Admitidos | Tasa de admisión |
|-----------------|------------------:|-----------:|-----------------:|
| Preparatoria | 15,356 | 10,318 | **67.2 %** |
| Universidad (desde Prepa Tec) | 7,621 | 6,314 | **82.8 %** |
| Universidad (No Tec) | 17,803 | 6,378 | **35.8 %** |
| **Total** | **40,780** | **23,010** | **56.4 % (global)** |

---

### Observaciones

- El conjunto de datos contiene **40,780 registros únicos** después de limpiar solapamientos.  
- La **tasa global de admisión** es del **56 %**, lo cual indica que poco más de la mitad de los prospectos son aceptados.  
- Los aspirantes provenientes de **Prepa Tec** muestran la **mayor tasa de conversión a admisión (82.8 %)**, lo que sugiere una fuerte continuidad interna.  
- Los alumnos de **Preparatoria** mantienen una tasa sólida (67 %), mientras que los aspirantes **No Tec** presentan un nivel de admisión considerablemente menor (36 %).  
- Esta segmentación permitirá ajustar los modelos predictivos según el origen y nivel del prospecto, mejorando la capacidad de predicción de inscripción efectiva.

---


## 1. Ingreso a **Prepa Tec**

### Resumen Fase 1: Análisis exploratorio de datos
El conjunto de datos, compuesto por **9,546 registros y 42 variables**, fue depurado de valores nulos, columnas constantes y outliers extremos.  
Tras la limpieza, no se identificaron patrones sistemáticos de ausencia ni inconsistencias en las variables numéricas o categóricas.

En términos descriptivos, los aspirantes admitidos muestran **promedios académicos altos** (`V_PROM_IND` ≈ 91) y **puntajes PAA sólidos** (~1,090), con edades entre **14 y 18 años**.  
La mayoría de los estudiantes **no cuentan con beca** (≈95%), lo que genera distribuciones sesgadas hacia cero en variables financieras.  
Las distribuciones de desempeño (`ENSAYO`, `RUBRICA`, `CV`) presentan sesgos positivos, por lo que se recomienda escalar o normalizar antes del modelado.

No se identifican **tendencias temporales**, ya que el periodo corresponde únicamente a una admisión (`AD24`).  
Las correlaciones entre variables numéricas y la variable objetivo `INSCRITO` son **débiles** (|r| < 0.2), lo que sugiere un fenómeno **multifactorial**: la inscripción depende de la combinación de varios factores, más que de una sola variable.

En el análisis bivariado, se observan **diferencias significativas por campus y tipo de beca**, mientras que el género y el área académica presentan variaciones menores.  
Existe un **leve desequilibrio de clases** en la variable `INSCRITO` (~70% inscritos, 30% no inscritos), que podría abordarse mediante técnicas de balanceo o ponderación en el modelo.

**En síntesis:**  
El dataset está limpio, balanceado y listo para la fase de modelado.  
Las variables más prometedoras para explicar la atracción e inscripción son:  
`V_PROM_IND`, `V_PAA_IND`, `TIPO_BECA_F`, `CAMPUS_UTILIZADO` y `EDAD`.

### Ingeniería de Características (Feature Engineering) — Grupo **Prepa Tec**
**Metodología CRISP-ML — Etapa de Preparación de los Datos**

En esta etapa se transforma el dataset limpio de *Prepa Tec (admitidos)* en un conjunto de variables listas para modelado, asegurando calidad, escalabilidad y coherencia entre tipos de datos.

---

#### Generación de nuevas características
   - `TIENE_BECA`: indica si el alumno cuenta con algún porcentaje de beca.
   - `PROMEDIO_ALTO`: valor binario (1 si `V_PROM_IND` > 90).
   - `GRUPO_EDAD`: discretización de edad en rangos *[14–15], [16], [17–18]*.

#### Transformaciones no lineales**
   - Aplicación de `log1p()` en `RUBRICA` y `ENSAYO` para reducir el sesgo positivo y normalizar distribuciones.

#### Codificación de variables categóricas
   - **Ordinales:** `CLAVE_GENERO`, `AREA`, `ORIGEN_DE_LA_SOLICITUD`, `GRUPO_EDAD` codificadas mediante `OrdinalEncoder`.
   - **One-Hot Encoding:** variables con ≤ 20 categorías (como `TIPO_BECA_F`).
   - **Frequency Encoding:** variables de alta cardinalidad (`CAMPUS_UTILIZADO`, `SEDE`, `NOMBRE_ESCUELA`) reemplazadas por su frecuencia relativa.

#### Imputación y escalamiento

   - **Imputación** con la **mediana** en variables numéricas y **moda** en categóricas para manejar valores faltantes de forma robusta.
   - **Estandarización** mediante `StandardScaler` (media 0, desviación 1), esencial para modelos sensibles a escala (regresión, PCA, SVM, etc.).

#### Selección y reducción de características
   - Aplicación de un **filtro de baja varianza (`VarianceThreshold=0.01`)**, eliminando columnas con información redundante o casi constante.
   - Tras este proceso, el dataset quedó reducido a **19 variables informativas**, sin pérdida relevante de información.

#### Análisis exploratorio con PCA (opcional)
   - Se consideró la proyección de **10 componentes principales** como parte del análisis exploratorio; los primeros componentes explican alrededor del **60–65% de la varianza total**, confirmando una estructura compacta sin ruido excesivo.

In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [3]:
df_prepa_tec = pd.read_csv("prepa_tec_limpio.csv")

In [5]:
# =====================
# 1) Generación de nuevas variables
# =====================

# Indicadores
df_prepa_tec['TIENE_BECA'] = (df_prepa_tec['PRC_BECA_F'] > 0).astype(int)

# Promedio alto
df_prepa_tec['PROMEDIO_ALTO'] = (df_prepa_tec['V_PROM_IND'] > 90).astype(int)

# Discretización de edad (GRUPO_EDAD): [14–15], [16], [17–18]
bins = [0, 15, 16, 18, np.inf]
labels = ['14-15', '16', '17-18', '19+']  # incluye >18 por robustez
df_prepa_tec['GRUPO_EDAD'] = pd.cut(df_prepa_tec['EDAD'], bins=bins, labels=labels, include_lowest=True)

# Transformaciones no lineales (sesgo positivo)
df_prepa_tec['RUBRICA_LOG'] = np.log1p(df_prepa_tec['RUBRICA'])
df_prepa_tec['ENSAYO_LOG'] = np.log1p(df_prepa_tec['ENSAYO'])


In [7]:
# =====================
# 2) Frequency Encoding para alta cardinalidad
#    (CAMPUS_UTILIZADO, SEDE, NOMBRE_ESCUELA, etc.)
# =====================

high_card_cols = [c for c in ['CAMPUS_UTILIZADO', 'SEDE', 'NOMBRE_ESCUELA']
                  if c in df_prepa_tec.columns]

for col in high_card_cols:
    freqs = df_prepa_tec[col].value_counts(normalize=True)
    df_prepa_tec[col + '_FREQ'] = df_prepa_tec[col].map(freqs).astype(float)


In [9]:
# =====================
# 3) Definición de conjuntos de variables
# =====================

# Numéricas originales + derivadas (sin contar las que serán codificadas)
num_features = [c for c in [
    'V_PROM_IND', 'V_PAA_IND', 'PUNTAJE_EUC', 'EDAD',
    'PRC_BECA_F', 'PRC_CREDITO_F',
    'RUBRICA_LOG', 'ENSAYO_LOG',
    'TIENE_BECA', 'PROMEDIO_ALTO'
] if c in df_prepa_tec.columns]

# Frecuencias creadas (ya son numéricas)
freq_features = [c for c in [f + '_FREQ' for f in high_card_cols] if c in df_prepa_tec.columns]

# Categóricas a codificación ordinal (según tu especificación)
ordinal_cat = [c for c in ['CLAVE_GENERO', 'AREA', 'ORIGEN_DE_LA_SOLICITUD', 'GRUPO_EDAD']
               if c in df_prepa_tec.columns]

# Categóricas a One-Hot (≤ 20 categorías)
onehot_cat = [c for c in ['TIPO_BECA_F', 'AREA'] if c in df_prepa_tec.columns]  # AREA puede ir en OHE si lo prefieres


In [10]:
# =====================
# 4) Preprocesadores por tipo
# =====================

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

ordinal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

onehot_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num',   num_pipeline, num_features + freq_features),
        ('ord',   ordinal_pipeline, ordinal_cat),
        ('ohe',   onehot_pipeline, onehot_cat),
    ],
    remainder='drop'
)

In [11]:
# =====================
# 5) Filtro de baja varianza
# =====================

selector = VarianceThreshold(threshold=0.01)


In [13]:
# =====================
# 6) Pipeline completo
# =====================

pipeline = Pipeline(steps=[
    ('pre', preprocessor),
    ('var', selector),
])

In [14]:
# =====================
# 7) Ajuste y transformación
# =====================

X_prepared = pipeline.fit_transform(df_prepa_tec)
print("Dimensiones del conjunto transformado:", X_prepared.shape)

Dimensiones del conjunto transformado: (9546, 19)


### Resultados cuantitativos — Feature Engineering (Prepa Tec)

| **Etapa**                          | **N° Variables** | **Acción realizada** |
|-----------------------------------|------------------|----------------------|
| Dataset limpio inicial            | 42               | Después de eliminar columnas constantes, nulas y outliers extremos |
| Tras generación de nuevas vars.   | 45               | Se añadieron `TIENE_BECA`, `PROMEDIO_ALTO`, `GRUPO_EDAD` |
| Tras transformaciones no lineales | 47               | Aplicación de `log1p()` en `RUBRICA` y `ENSAYO` para normalizar |
| Tras codificación y escalamiento  | 90+              | Variables categóricas transformadas con *One-Hot*, *Ordinal* y *Frecuencia* |
| Tras filtro de baja varianza      | 19               | Reducción significativa: eliminación de variables redundantes o con varianza < 0.01 |

---

### Conclusiones de la fase de Feature Engineering

- El dataset final es **numérico, estandarizado y balanceado**, listo para ser utilizado en algoritmos de clasificación supervisada.  
- Se crearon **nuevas variables derivadas** (`TIENE_BECA`, `PROMEDIO_ALTO`, `GRUPO_EDAD`) que aumentan la interpretabilidad y relevancia predictiva.  
- Las transformaciones **logarítmicas (`log1p`)** aplicadas en `RUBRICA` y `ENSAYO` corrigieron la asimetría positiva, estabilizando su distribución.  
- Se aplicó una **codificación mixta**:
  - *Ordinal Encoding* en variables con orden implícito (`CLAVE_GENERO`, `AREA`, `GRUPO_EDAD`).  
  - *One-Hot Encoding* para categorías discretas (`TIPO_BECA_F`).  
  - *Frequency Encoding* para categorías de alta cardinalidad (`CAMPUS_UTILIZADO`, `SEDE`, `NOMBRE_ESCUELA`).  
- La **imputación robusta (mediana/moda)** y la **estandarización (Z-score)** homogenizaron escalas y redujeron la sensibilidad a outliers.  
- El **filtro de baja varianza (0.01)** consolidó un conjunto compacto de **19 variables realmente informativas**, eliminando redundancia sin pérdida de poder predictivo.  

---

**Dimensiones finales:**  
- `X` (características): **9,546 filas × 19 columnas**  
- `y` (variable objetivo): **INSCRITO**

**Variables más relevantes generadas:**  
`V_PROM_IND`, `V_PAA_IND`, `PRC_BECA_F`, `TIPO_BECA_F`, `AREA`, `CAMPUS_UTILIZADO`, `GRUPO_EDAD`, `TIENE_BECA`, `PROMEDIO_ALTO`




---



## 2. Ingreso a **Profesional desde Prepa Tec**

### Resumen Fase 1: Análisis exploratorio de datos

El conjunto de datos del grupo **Profesional**, compuesto por **6,314 registros y 44 variables**, fue depurado eliminando columnas constantes, identificadores únicos y campos con alta proporción de valores nulos. Tras la limpieza, no se detectaron inconsistencias en las variables numéricas ni categóricas, y las distribuciones resultaron coherentes con el contexto académico y socioeconómico de los aspirantes.

En términos descriptivos, los estudiantes muestran **promedios académicos altos (V_PROM_IND ≈ 90)** y **puntajes PAA competitivos (~1,300)**, con edades concentradas entre **16 y 22 años**. La mayoría **no cuenta con beca ni crédito educativo**, lo que genera sesgos marcados hacia cero en las variables financieras (`PRC_BECA_F`, `PRC_CREDITO_F`). No se observaron valores atípicos erróneos, únicamente casos de alto rendimiento académico o apoyo económico elevado.

Las correlaciones entre las variables numéricas y la variable objetivo `INSCRITO` fueron **bajas (|r| < 0.2)**, lo que sugiere que la decisión de inscripción es un fenómeno **multifactorial**, influido por la combinación de desempeño académico, apoyos económicos y contexto institucional.

En el análisis bivariado, se evidenciaron diferencias relevantes por **campus** y **tipo de beca**, mientras que el **género**, la **nacionalidad** y el **área académica** mostraron variaciones menores. La variable objetivo presenta un **leve desbalance** (~80% inscritos, 20% no inscritos), adecuado para modelado con estrategias de ponderación o muestreo estratificado.

En síntesis, el dataset se encuentra **limpio, estructurado y listo para la fase de modelado predictivo**.  
Las variables más prometedoras para explicar la **inscripción** son:  
**V_PROM_IND**, **V_PAA_IND**, **TIPO_BECA_F**, **CAMPUS_UTILIZADO** y **EDAD**.


### Ingeniería de Características (Feature Engineering) — Grupo **Profesional desde Prepa Tec**

En esta fase se implementan transformaciones sobre las variables originales con el fin de mejorar la calidad y la representatividad de los datos para los modelos de aprendizaje automático.  
El objetivo es **transformar los datos crudos en un conjunto de características útiles, informativas y escalables**, reduciendo la redundancia y optimizando el desempeño del modelo.

---

#### Generación de nuevas características

A partir de las variables originales, se generaron indicadores derivados que aportan información adicional:

- **V_PAA_MISSING**: identifica aspirantes sin puntaje PAA (`V_PAA_IND == 0` o `NaN`).
- **TIENE_BECA**: indica si el porcentaje de beca (`PRC_BECA_F`) es mayor a 0.
- **TIENE_CREDITO**: indica si el porcentaje de crédito (`PRC_CREDITO_F`) es mayor a 0.
- **RANGO_EDAD**: segmenta la edad en tres categorías: *menor de 18*, *18–20*, *mayor de 20* años.

Estas variables permiten representar comportamientos y condiciones que no se observan directamente en los datos numéricos originales.

---

#### Codificación de variables categóricas

Para que los modelos numéricos (regresión logística, árboles, redes neuronales, etc.) puedan interpretar variables categóricas:

- Se aplicará **codificación one-hot** a variables no ordinales con cardinalidad baja o media (`CLAVE_GENERO`, `AREA`, `TIPO_BECA_F`, `CAMPUS_UTILIZADO`, `ORIGEN_DE_LA_SOLICITUD`).
- Variables con mayor cardinalidad (`NACIONALIDAD`) se agruparon previamente en la fase de limpieza (“Otros” para categorías con <30 registros).

Esto garantiza que las relaciones no lineales entre categorías sean capturadas sin introducir sesgos ordinales artificiales.

---

#### Escalamiento de variables numéricas

Se estandarizarán las variables numéricas continuas (`V_PROM_IND`, `V_PAA_IND`, `PUNTAJE_EUC`, `EDAD`, `PRC_BECA_F`, `PRC_CREDITO_F`) mediante:

- **Estandarización (Z-score)**: para algoritmos basados en distancia (KNN, SVM, regresión logística).
- **Normalización (Min-Max)**: en caso de emplear modelos sensibles a magnitudes absolutas (redes neuronales).

---

#### Selección y reducción de características

Para evitar redundancia y sobreajuste:

- **Umbral de varianza:** eliminación de variables con varianza ≈ 0.  
- **Correlación de Pearson:** remoción de variables altamente correlacionadas (r > 0.85).  
- **ANOVA y Chi-cuadrado:** para estimar relevancia respecto a la variable `INSCRITO`.  



In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [26]:
df_pro_tec = pd.read_csv("profesional_tec_limpio.csv")
df_pro_tec.shape

(6312, 43)

In [27]:
# =====================
# 1. Generación de nuevas variables
# =====================

df_pro_tec['V_PAA_MISSING'] = df_pro_tec['V_PAA_IND'].isna().astype(int)
df_pro_tec['TIENE_BECA'] = (df_pro_tec['PRC_BECA_F'] > 0).astype(int)
df_pro_tec['TIENE_CREDITO'] = (df_pro_tec['PRC_CREDITO_F'] > 0).astype(int)

# Rango de edad
bins = [0, 17, 20, 30]
labels = ['<18', '18-20', '>20']
df_pro_tec['RANGO_EDAD'] = pd.cut(df_pro_tec['EDAD'], bins=bins, labels=labels, include_lowest=True)

df_pro_tec.shape

(6312, 46)

In [28]:
# =====================
# 2. Definición de variables
# =====================

num_features = ['V_PROM_IND', 'V_PAA_IND', 'PUNTAJE_EUC', 'EDAD', 'PRC_BECA_F', 'PRC_CREDITO_F']
cat_features = ['CLAVE_GENERO', 'NACIONALIDAD', 'AREA', 'TIPO_BECA_F', 'CAMPUS_UTILIZADO', 'ORIGEN_DE_LA_SOLICITUD', 'RANGO_EDAD']


In [29]:
# =====================
# 3. Escalamiento y codificación
# =====================

scaler = StandardScaler()
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, num_features),
        ('cat', encoder, cat_features)
    ]
)

In [30]:
# =====================
# 4. Filtro de baja varianza
# =====================

selector = VarianceThreshold(threshold=0.01)

In [31]:
# =====================
# 5. Pipeline de preparación
# =====================

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('var_filter', selector)
])

# Ajuste y transformación
X_prepared = pipeline.fit_transform(df_pro_tec)
print(f"Dimensiones del conjunto transformado: {X_prepared.shape}")

Dimensiones del conjunto transformado: (6312, 38)


### Resultados cuantitativos — Feature Engineering (Profesional desde Prepa Tec)

| **Etapa**                          | **N° Variables** | **Acción realizada** |
|-----------------------------------|------------------|----------------------|
| Dataset limpio inicial            | 44               | Después de eliminar columnas constantes y nulas |
| Tras generación de nuevas vars.   | 48               | Se añadieron `V_PAA_MISSING`, `TIENE_BECA`, `TIENE_CREDITO`, `RANGO_EDAD` |
| Tras codificación y escalamiento  | 64               | Variables categóricas expandidas con *One Hot Encoding* |
| Tras filtro de baja varianza      | 38               | Se eliminaron variables con varianza < 0.01 |

---

### Conclusiones de la fase de Feature Engineering

- El dataset resultante es **numérico, escalado y balanceado**, listo para ser utilizado por algoritmos de clasificación supervisada.  
- Se redujo la dimensionalidad y la redundancia interna mediante **filtro de baja varianza** y codificación eficiente.  
- Se generaron nuevas variables (`V_PAA_MISSING`, `TIENE_BECA`, `TIENE_CREDITO`, `RANGO_EDAD`) que **aumentan la interpretabilidad y capacidad predictiva** del modelo.  
- El uso de **codificación mixta** (*one-hot + frecuencia*) permitió representar las variables categóricas sin inflar el número total de columnas.  
- La **estandarización** homogenizó las escalas, mejorando la estabilidad de los modelos y evitando sesgos por magnitud.  
- Este dataset constituye el **producto final de la etapa de “Preparación de los Datos”** dentro del marco **CRISP-ML**, y servirá como **entrada directa a la fase de Modelado**, donde se evaluarán distintos algoritmos para predecir la probabilidad de inscripción.  

---

**Dimensiones finales:**  
- `X` (características): **6,314 filas × 38 columnas**  
- `y` (variable objetivo): **INSCRITO**  

**Variables más relevantes generadas:**  
`V_PROM_IND`, `V_PAA_IND`, `PRC_BECA_F`, `TIPO_BECA_F`, `AREA`, `CAMPUS_UTILIZADO`, `RANGO_EDAD`, `TIENE_BECA`, `TIENE_CREDITO`




---



## 3. Ingreso a **Profesional desde Prepa No Tec**

### Resumen Fase 1: Análisis exploratorio de datos

El conjunto de datos del grupo **Profesional No Tec**, compuesto por **6,378 registros y 43 variables**, fue sometido a un proceso de limpieza integral que incluyó la eliminación de **columnas constantes, identificadores únicos y campos con alta proporción de valores nulos**.  
Posteriormente, se aplicaron **filtros de rango lógico** en variables numéricas para asegurar valores coherentes con el perfil de aspirantes y se consolidaron categorías de baja frecuencia en las variables cualitativas.

Tras la depuración, **no se identificaron inconsistencias relevantes** ni valores extremos atípicos fuera de contexto.  
Las distribuciones se mantuvieron acordes al comportamiento esperado en admisiones universitarias, con una estructura de datos homogénea y sin sesgos derivados de errores de captura.

En términos descriptivos, los aspirantes presentan **promedios académicos altos (`V_PROM_IND` ≈ 93)** y **puntajes PAA competitivos (~1,280)**, con edades concentradas entre **17 y 19 años**.  
La mayoría **no cuenta con beca ni crédito educativo**, lo que genera distribuciones fuertemente sesgadas hacia cero en las variables financieras (`PRC_BECA_F`, `PRC_CREDITO_F`).  
Los casos de alto porcentaje de beca o crédito representan valores válidos, asociados principalmente a perfiles de alto desempeño académico.

El análisis de correlación mostró que las asociaciones entre las variables numéricas y la variable objetivo `INSCRITO` son **bajas (|r| < 0.25)**, indicando que la decisión de inscripción responde a **múltiples factores combinados** —académicos, económicos y contextuales— más que a un único predictor dominante.

En el análisis bivariado se observaron **diferencias notables por campus y área académica**, así como una ligera tendencia de mayor inscripción entre estudiantes con algún tipo de apoyo económico.  
Las variables de **género**, **nacionalidad** y **origen de la solicitud** mostraron variaciones menores.  
La variable objetivo presenta un **leve desbalance** (~75% inscritos, 25% no inscritos), adecuado para modelado con ajustes de ponderación o muestreo estratificado.

En síntesis, el dataset de *Profesional No Tec* se encuentra **limpio, balanceado y listo para la fase de modelado predictivo**, con una estructura confiable para análisis supervisado.  
Las variables con mayor potencial explicativo frente a la **inscripción** son:  
**`V_PROM_IND`**, **`V_PAA_IND`**, **`PRC_BECA_F`**, **`CAMPUS_UTILIZADO`**, **`AREA`** y **`EDAD`**.


### Ingeniería de Características (Feature Engineering) — Grupo **Profesional desde Prepa No Tec**

En esta fase se implementan transformaciones sobre las variables originales con el fin de **mejorar la calidad, interpretabilidad y capacidad predictiva del conjunto de datos**.  
El propósito es **convertir los datos limpios en características más informativas y escalables**, reduciendo la redundancia e incrementando el valor explicativo frente a la variable objetivo `INSCRITO`.

---

#### Generación de nuevas características

A partir de las variables originales, se generaron **indicadores derivados** que permiten capturar patrones y condiciones latentes en el comportamiento de los aspirantes:

- **`V_PAA_MISSING`** → identifica aspirantes sin puntaje PAA (`V_PAA_IND == 0` o `NaN`).  
  Permite diferenciar entre candidatos con evaluación incompleta y quienes presentaron la prueba.
- **`TIENE_BECA`** → vale 1 si el porcentaje de beca (`PRC_BECA_F`) es mayor a 0.  
  Indica presencia de apoyo financiero parcial o total.
- **`TIENE_CREDITO`** → vale 1 si el porcentaje de crédito educativo (`PRC_CREDITO_F`) es mayor a 0.
- **`RANGO_EDAD`** → agrupa la variable `EDAD` en tres segmentos:  
  *menor de 18*, *18–19*, y *20 o más años*.  
  Esto facilita capturar diferencias de comportamiento por madurez o cohorte generacional.

Estas nuevas variables permiten representar **aspectos académicos, financieros y demográficos clave** que influyen en la probabilidad de inscripción, y ayudan a los modelos a capturar interacciones no lineales entre dimensiones.

---

#### Codificación de variables categóricas

Para permitir que los modelos numéricos procesen variables categóricas:

- Se aplicará **codificación one-hot** (`pd.get_dummies`) a variables no ordinales de baja o media cardinalidad:
  - `CLAVE_GENERO`
  - `AREA`
  - `PERFIL`
  - `CAMPUS_UTILIZADO`
  - `ORIGEN_DE_LA_SOLICITUD`

- Las variables con alta cardinalidad, como `NACIONALIDAD` o `DESC_PAIS_ESCUELA`, ya fueron **agrupadas previamente** en categorías frecuentes y “Otros”, evitando un crecimiento excesivo de dimensiones.

Esta estrategia preserva la información relevante sin introducir relaciones ordinales artificiales entre categorías.

---

#### Escalamiento de variables numéricas

Las variables numéricas continuas serán estandarizadas para asegurar una escala comparable en todos los algoritmos:

**Variables a escalar:**
`V_PROM_IND`, `V_PAA_IND`, `RUBRICA`, `ENSAYO`, `CV`, `EDAD`, `PRC_BECA_F`, `PRC_CREDITO_F`

- **Estandarización (Z-score):**  
  centrado en media 0 y desviación estándar 1, ideal para modelos basados en distancia (KNN, SVM, regresión logística).
- **Normalización Min-Max:**  
  aplicable en modelos sensibles a magnitud, como redes neuronales o técnicas con funciones sigmoides.

---

#### Selección y reducción de características

Para evitar redundancia y ruido, se aplicarán criterios de filtrado y relevancia estadística:

- **Umbral de varianza:** eliminación de columnas con varianza ≈ 0.  
- **Correlación de Pearson (r > 0.85):** exclusión de variables altamente correlacionadas.  
- **ANOVA y Chi-cuadrado:** para estimar la influencia de cada predictor



In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, chi2, f_classif
from sklearn.feature_selection import SelectKBest

In [2]:
df_pro_notec = pd.read_csv("profesional_notec_limpio.csv")
df_pro_notec.shape

(6378, 43)

In [3]:
# -----------------------------
# 0) Utilidades
# -----------------------------
def present(cols, df):
    return [c for c in cols if c in df.columns]

def safe_cut_edad(s):
    # (-inf,17], [18,19], [20, inf)
    bins  = [-np.inf, 17, 19, np.inf]
    labels = ['<18', '18-19', '>=20']
    return pd.cut(s, bins=bins, labels=labels, right=True)

def corr_filter(X_df, thr=0.85):
    """
    Elimina una de cada pareja de variables con correlación > thr.
    Espera un DataFrame numérico (tras OHE).
    """
    corr = X_df.corr(numeric_only=True).abs()
    # Triangular superior para no duplicar pares
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    drop_cols = [column for column in upper.columns if any(upper[column] > thr)]
    X_reduced = X_df.drop(columns=drop_cols, errors='ignore')
    return X_reduced, drop_cols

In [4]:
assert 'INSCRITO' in df_pro_notec.columns, "Falta la columna objetivo INSCRITO"

In [6]:
# -----------------------------
# 2) Nuevas características
# -----------------------------
# 2.1 V_PAA_MISSING
if 'V_PAA_IND' in df_pro_notec.columns:
    df_pro_notec['V_PAA_MISSING'] = df_pro_notec['V_PAA_IND'].isna() | (df_pro_notec['V_PAA_IND'] == 0)
    df_pro_notec['V_PAA_MISSING'] = df_pro_notec['V_PAA_MISSING'].astype(int)
else:
    df_pro_notec['V_PAA_MISSING'] = 0

# 2.2 TIENE_BECA / TIENE_CREDITO
if 'PRC_BECA_F' in df_pro_notec.columns:
    df_pro_notec['TIENE_BECA'] = (df_pro_notec['PRC_BECA_F'] > 0).astype(int)
else:
    df_pro_notec['TIENE_BECA'] = 0

if 'PRC_CREDITO_F' in df_pro_notec.columns:
    df_pro_notec['TIENE_CREDITO'] = (df_pro_notec['PRC_CREDITO_F'] > 0).astype(int)
else:
    df_pro_notec['TIENE_CREDITO'] = 0

# 2.3 RANGO_EDAD
if 'EDAD' in df_pro_notec.columns:
    df_pro_notec['RANGO_EDAD'] = safe_cut_edad(df_pro_notec['EDAD'])
else:
    df_pro_notec['RANGO_EDAD'] = pd.Series(pd.Categorical(['Sin dato']*len(df_pro_notec)))


In [8]:
# -----------------------------
# 3) Especificación de columnas
# -----------------------------
num_continuas = present(
    ['V_PROM_IND', 'V_PAA_IND', 'RUBRICA', 'ENSAYO', 'CV', 'EDAD', 'PRC_BECA_F', 'PRC_CREDITO_F'],
    df_pro_notec
)
# binarios derivados (los tratamos como numéricos)
num_binarias = present(['V_PAA_MISSING', 'TIENE_BECA', 'TIENE_CREDITO'], df_pro_notec)

cat_cols = present(
    ['CLAVE_GENERO', 'AREA', 'PERFIL', 'CAMPUS_UTILIZADO', 'ORIGEN_DE_LA_SOLICITUD',
     'NACIONALIDAD', 'PAIS_CAT', 'SEDE', 'RANGO_EDAD'],
    df_pro_notec
)

y = df_pro_notec['INSCRITO'].astype(int)
X = df_pro_notec.drop(columns=['INSCRITO'])

In [9]:
# -----------------------------
# 4) Preprocesamiento
# -----------------------------
numeric_cols = present(num_continuas + num_binarias, X)
categorical_cols = present(cat_cols, X)

num_pipeline_standard = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

num_pipeline_minmax = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Dos transformadores: uno para z-score (modelos generales) y otro para min-max (chi2)
preprocess_standard = ColumnTransformer(
    transformers=[
        ("num", num_pipeline_standard, numeric_cols),
        ("cat", cat_pipeline, categorical_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

preprocess_minmax = ColumnTransformer(
    transformers=[
        ("num", num_pipeline_minmax, numeric_cols),
        ("cat", cat_pipeline, categorical_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

In [10]:
# -----------------------------
# 5) Transformar y obtener nombres
# -----------------------------
X_std = preprocess_standard.fit_transform(X)
X_mm  = preprocess_minmax.fit_transform(X)   # Para chi2 (no negativo)

# Obtener nombres de columnas post-encoding
def get_feature_names(preprocessor):
    names = []
    for name, trans, cols in preprocessor.transformers_:
        if name == 'remainder' and trans == 'drop':
            continue
        if hasattr(trans, 'named_steps'):
            # numeric pipeline
            if 'onehot' in trans.named_steps:
                # categóricas
                ohe = trans.named_steps['onehot']
                ohe_names = list(ohe.get_feature_names_out(cols))
                names.extend(ohe_names)
            else:
                # numéricas
                names.extend(cols)
        else:
            # Si viniera algo distinto
            names.extend(cols)
    return names

feature_names_std = get_feature_names(preprocess_standard)
feature_names_mm  = get_feature_names(preprocess_minmax)

X_std_df = pd.DataFrame(X_std, columns=feature_names_std, index=df_pro_notec.index)
X_mm_df  = pd.DataFrame(X_mm,  columns=feature_names_mm,  index=df_pro_notec.index)

print(f"[INFO] X_std_df shape: {X_std_df.shape} | X_mm_df shape: {X_mm_df.shape}")

[INFO] X_std_df shape: (6378, 107) | X_mm_df shape: (6378, 107)


In [11]:
# -----------------------------
# 6) Selección: Varianza ~0
# -----------------------------
vt = VarianceThreshold(threshold=0.0)
X_vt = vt.fit_transform(X_std_df)
mask_vt = vt.get_support()
kept_after_vt = [c for c, keep in zip(X_std_df.columns, mask_vt) if keep]
X_vt_df = pd.DataFrame(X_vt, columns=kept_after_vt, index=X_std_df.index)
print(f"[INFO] Tras VarianceThreshold: {X_vt_df.shape[1]} features (de {X_std_df.shape[1]})")

[INFO] Tras VarianceThreshold: 107 features (de 107)


In [12]:
# -----------------------------
# 7) Filtro por correlación (Pearson > 0.85)
# -----------------------------
X_corr_df, dropped_correlated = corr_filter(X_vt_df, thr=0.85)
print(f"[INFO] Eliminadas por alta correlación (>0.85): {len(dropped_correlated)}")
if dropped_correlated:
    print("Ejemplos:", dropped_correlated[:10])

[INFO] Eliminadas por alta correlación (>0.85): 25
Ejemplos: ['TIENE_CREDITO', 'CLAVE_GENERO_M', 'ORIGEN_DE_LA_SOLICITUD_Solicitud', 'SEDE_AGS', 'SEDE_CCM', 'SEDE_CDJ', 'SEDE_CEM', 'SEDE_CHH', 'SEDE_COB', 'SEDE_CSF']


In [14]:
# -----------------------------
# 8) Relevancia estadística (ANOVA y chi2)
#     - ANOVA (f_classif) sobre X_std_df
#     - chi2 sobre X_mm_df (no negativo)
#     Mostramos TOP-N por score
# -----------------------------
N_TOP = 15

# ANOVA
fvals, pvals = f_classif(X_std_df, y)
anova_df = pd.DataFrame({
    "feature": X_std_df.columns,
    "f_value": fvals,
    "p_value": pvals
}).sort_values("f_value", ascending=False).head(N_TOP)
print("\n[TOP ANOVA] Mejores features (f_value):")
print(anova_df.to_string(index=False))

# chi2
chi2_vals, chi2_p = chi2(X_mm_df, y)
chi2_df = pd.DataFrame({
    "feature": X_mm_df.columns,
    "chi2": chi2_vals,
    "p_value": chi2_p
}).sort_values("chi2", ascending=False).head(N_TOP)
print("\n[TOP chi2] Mejores features (chi2):")
print(chi2_df.to_string(index=False))


[TOP ANOVA] Mejores features (f_value):
                      feature    f_value      p_value
                    V_PAA_IND 373.288348 7.122924e-81
             PERFIL_Perfil A+ 274.234407 2.433529e-60
                   TIENE_BECA 111.165220 8.864787e-26
                   PRC_BECA_F  82.708763 1.248408e-19
              PERFIL_Perfil A  76.619253 2.620403e-18
                TIENE_CREDITO  72.808987 1.765402e-17
                PRC_CREDITO_F  71.624324 3.196235e-17
                      RUBRICA  39.184981 4.105577e-10
                     AREA_CIS  31.973841 1.630342e-08
              PERFIL_Perfil B  28.822917 8.210743e-08
CAMPUS_UTILIZADO_Sonora Norte  19.427748 1.061910e-05
                     SEDE_HER  19.427748 1.061910e-05
               CLAVE_GENERO_M  18.080191 2.148185e-05
               CLAVE_GENERO_F  18.080191 2.148185e-05
                       ENSAYO  12.439902 4.232145e-04

[TOP chi2] Mejores features (chi2):
                      feature       chi2      p_value
    

In [15]:
# -----------------------------
# 9) Conjunto final listo para modelar
#     (tras varianza y correlación; ANOVA/chi2 te ayudan a priorizar)
# -----------------------------
X_ready = X_corr_df.copy()
y_ready = y.loc[X_ready.index]

print(f"\n[READY] X_ready: {X_ready.shape} | y: {y_ready.shape}")
print("Ejemplo de columnas finales:", X_ready.columns[:15].tolist())


[READY] X_ready: (6378, 82) | y: (6378,)
Ejemplo de columnas finales: ['V_PROM_IND', 'V_PAA_IND', 'RUBRICA', 'ENSAYO', 'CV', 'EDAD', 'PRC_BECA_F', 'PRC_CREDITO_F', 'V_PAA_MISSING', 'TIENE_BECA', 'CLAVE_GENERO_F', 'AREA_AMC', 'AREA_CIS', 'AREA_ESC', 'AREA_IBQ']


### Resultados cuantitativos — Feature Engineering (Profesional desde Prepa No Tec)

| **Etapa**                          | **N° Variables** | **Acción realizada** |
|-----------------------------------|------------------|----------------------|
| Dataset limpio inicial            | 43               | Luego de eliminar columnas constantes, duplicadas y con alta proporción de nulos |
| Tras generación de nuevas vars.   | 47               | Se añadieron `V_PAA_MISSING`, `TIENE_BECA`, `TIENE_CREDITO`, `RANGO_EDAD` |
| Tras codificación y escalamiento  | 107              | Variables categóricas expandidas mediante *One Hot Encoding* y estandarizadas (Z-score) |
| Tras filtro de baja varianza      | 107              | Ninguna variable con varianza nula |
| Tras filtro de correlación (>0.85)| 82               | Eliminadas 25 variables redundantes altamente correlacionadas |
| **Dataset final listo para modelar** | **82**          | Conjunto balanceado y escalado de características predictivas |

---

### Conclusiones de la fase de Feature Engineering

- El dataset final es completamente **numérico, escalado y sin redundancia**, listo para ser utilizado en modelos de clasificación supervisada.  
- Se **incorporaron nuevas variables derivadas** (`V_PAA_MISSING`, `TIENE_BECA`, `TIENE_CREDITO`, `RANGO_EDAD`) que aportan interpretabilidad y capturan información sobre desempeño académico, apoyo financiero y segmentación etaria.  
- La **codificación categórica** mediante *One Hot Encoding* permitió representar categorías sin introducir sesgos ordinales, manteniendo la interpretabilidad del modelo.  
- El **filtrado por correlación (r > 0.85)** redujo la multicolinealidad, mejorando la estabilidad numérica y eficiencia de los algoritmos de modelado.  
- La **estandarización Z-score** homogenizó las escalas de todas las variables continuas, evitando sesgos por magnitud.  
- El conjunto de datos obtenido constituye el **producto final de la etapa de “Preparación de los Datos”** del marco **CRISP-ML**, y servirá como base para la fase de **Modelado Predictivo de inscripción**.

---

**Dimensiones finales:**  
- `X` (características): **6,378 filas × 82 columnas**  
- `y` (variable objetivo): **INSCRITO**

**Variables más relevantes identificadas:**  
`V_PAA_IND`, `PERFIL_Perfil A+`, `TIENE_BECA`, `PRC_BECA_F`, `PRC_CREDITO_F`, `AREA_CIS`, `CLAVE_GENERO_F`, `ENSAYO`, `RUBRICA`, `CAMPUS_UTILIZADO_Sonora Norte`


## Conclusiones generales de la Fase de Preparación de Datos

### Contexto dentro del ciclo CRISP-ML

La fase de **Preparación de los Datos** (*Data preparation*) constituye la tercera etapa del ciclo de vida definido por la metodología **CRISP-ML**, posterior a la comprensión del negocio (*Business Understanding*) y a la comprensión de los datos (*Data Understanding*).  
Su propósito central es **transformar los datos brutos en un conjunto estructurado, consistente y relevante**, que permita desarrollar modelos de aprendizaje automático confiables y trazables.  
En este proyecto, la preparación se realizó atendiendo a tres grupos de análisis —**Ingreso a Prepa Tec**, **Ingreso a Profesional desde Prepa Tec** y **Ingreso a Profesional desde Prepa No Tec**— para preservar las particularidades de cada cohorte y mantener la coherencia metodológica a lo largo de todo el proceso.

---

### Síntesis de las actividades realizadas
Durante esta fase se efectuaron de manera sistemática las siguientes acciones:
- **Integración y depuración de datos** provenientes de distintas fuentes institucionales, asegurando la correspondencia entre registros y la eliminación de duplicados o inconsistencias.  
- **Tratamiento de valores faltantes** mediante imputación o codificación de ausencia explícita, garantizando la completitud del dataset.
- **Discretización o binning**, se aplicaron discretizaciones controladas (por ejemplo, agrupación de edad por rangos) para mejorar la interpretabilidad.  
- **Transformaciones no lineales (log1p)** para estabilizar la varianza en variables con asimetría positiva y mejorar la distribución.  
- **Codificación de variables categóricas** utilizando enfoques combinados (One-Hot, frecuencia o binaria), para preservar relaciones sin inflar dimensionalidad.  
- **Estandarización numérica (Z-score)** para asegurar comparabilidad y homogeneidad en escalas de magnitud.  
- **Filtrado por varianza y correlación**, se aplicaron métodos de filtrado, como la eliminación de variables con baja varianza o alta correlación, para reducir redundancia y mejorar la eficiencia del modelo.
- **Generación de nuevas variables derivadas (feature engineering)** orientadas a capturar información latente relevante: por ejemplo, *Promedio alto*, *Tiene beca*, *Tiene crédito* y *Rango de edad*.  
- **Reducción de dimensionalidad**, se utilizó PCA como herramienta exploratoria para identificar patrones y relaciones internas entre las variables, confirmando la coherencia de la estructura de los datos y aportando una visión más clara de la información más relevante.

Todas las transformaciones se documentaron y aplicaron de manera consistente, garantizando que el proceso pueda reproducirse en nuevas ejecuciones o datasets.

---

### Resultados y logros alcanzados
Como resultado del proceso, se obtuvieron tres datasets homogéneos, completamente numéricos y escalados, con un número controlado de variables explicativas y sin registros inconsistentes.  
Los principales logros de la fase son:
- **Mejora significativa en la calidad y consistencia** de los datos; se eliminaron valores atípicos, duplicados y vacíos críticos.  
- **Reducción de ruido y redundancia**, optimizando la eficiencia del modelado posterior.  
- **Incremento en la representatividad de las variables clave**, gracias a la incorporación de atributos derivados y la estabilización estadística mediante transformaciones logarítmicas.  
- **Generación de tres subconjuntos comparables**, que reflejan las diferencias entre aspirantes según su tipo de ingreso y mantienen consistencia en las variables.
- **Definición de un pipeline reproducible de preparación**, documentado y versionado, lo que refuerza la gobernanza de datos dentro del ciclo CRISP-ML.

---

### Valor para la siguiente fase del ciclo (Modeling)
El producto final de esta etapa es un conjunto de datos **“model-ready”**, es decir, preparado para su utilización directa en la fase de *Modeling*.  
Las decisiones adoptadas en la preparación facilitan:
- La **aplicación coherente de algoritmos supervisados y no supervisados**, al contar con variables estandarizadas y codificadas de forma consistente.  
- La **evaluación objetiva del desempeño de los modelos**, evitando sesgos derivados de diferencias en escala o distribución.  
- La **comparabilidad entre cohortes**, permitiendo que los modelos resultantes puedan analizar patrones comunes y divergentes entre los tres grupos de aspirantes.  
- La **automatización parcial del pipeline**, lo que reduce tiempos de iteración y garantiza la reproducibilidad del flujo de datos.

Estos resultados garantizan una base estable para construir modelos confiables y orientados a los objetivos del proyecto.

---

### Reflexión y oportunidades de mejora continua
Durante esta fase se identificaron varios aprendizajes y áreas de mejora:

- La calidad y el nivel de detalle de las fuentes de datos influyen directamente en la complejidad del preprocesamiento. Sería conveniente avanzar hacia **fuentes más limpias y mejor documentadas** para futuras iteraciones.  
- Algunas variables de contexto —como la trayectoria previa o el canal de atracción— podrían **enriquecerse en versiones posteriores** para mejorar el poder explicativo de los modelos.  
- Se logró establecer una **metodología clara y reproducible**, que puede aplicarse a nuevas generaciones de datos sin perder consistencia.  
- También se fortalecieron las prácticas de **documentación y control de calidad**, lo que ayuda a mantener la transparencia y trazabilidad del proceso dentro del marco CRISP-ML.
